In [ ]:
# Add two empty columns to the patient DataFrame
df_pat['Distance to nearest dispensary'] = ''
df_pat['Zipcode of nearest dispensary'] = ''

# Function to calculate distance using geopy
def calculate_distance(row):
    patient_coords = (row['Latitude'], row['Longitude'])
    distances = df_dispo.apply(lambda x: distance(patient_coords, (x['Latitude'], x['Longitude'])).miles, axis=1)
    nearest_disp_index = distances.idxmin()
    nearest_disp_zipcode = df_dispo.loc[nearest_disp_index, 'Zipcode']
    nearest_disp_distance = distances[nearest_disp_index]
    return row.Index, nearest_disp_distance, nearest_disp_zipcode

# Calculate nearest neighbor for each patient using parallel processing
def process_patient(row):
    nearest_distance, nearest_zipcode = calculate_distance(row)
    return nearest_distance, nearest_zipcode

# Number of threads to use
num_threads = 12

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process patients in parallel
    results = executor.map(process_patient, df_pat.iterrows())

    # Update the patient DataFrame with results
    for idx, (nearest_distance, nearest_zipcode) in results:
        df_pat.loc[idx, 'Distance to nearest dispensary'] = nearest_distance
        df_pat.loc[idx, 'Zipcode of nearest dispensary'] = nearest_zipcode

# Save the patient DataFrame to a CSV file
df_pat.to_csv('Patient_Analysis.csv', index=False)


In [ ]:
# Step 1: Select lat and long columns from dataframe
lat_long_data = df_dispo[['Latitude', 'Longitude']]

# Convert to geodataframe with CRS EPSG 4269
lat_long_data = gpd.GeoDataFrame(lat_long_data, geometry=gpd.points_from_xy(lat_long_data.Longitude, lat_long_data.Latitude), crs=4269)

# Step 2: Load zip code boundary shapefile
zip_code_boundaries = gpd.read_file('tl_2017_42_place/tl_2017_42_place.shp')
print(zip_code_boundaries.head())

# Step 3: Spatial join
joined_data = gpd.sjoin(zip_code_boundaries, lat_long_data, how='inner', op='contains')

# Step 4: Aggregate counts
heatmap_data = joined_data.groupby('zip_code')['Latitude'].count().reset_index()

# Step 5: Plot the heatmap
fig, ax = plt.subplots(figsize=(10, 10))
zip_code_boundaries.plot(ax=ax, color='white', edgecolor='black')
heatmap_data.plot(column='latitude', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='black', legend=True)
plt.title('Heatmap of Latitude/Longitude Pairs by Zip Code')
plt.show()

  STATEFP PLACEFP   PLACENS    GEOID          NAME              NAMELSAD LSAD  \
0      42   47560  01215231  4247560  Markleysburg  Markleysburg borough   21   
1      42   37792  01215394  4237792         Jeddo         Jeddo borough   21   
2      42   48480  01215187  4248480         Media         Media borough   21   
3      42   76904  01215142  4276904    Titusville       Titusville city   25   
4      42   04136  01215635  4204136   Barkeyville   Barkeyville borough   21   

  CLASSFP PCICBSA PCINECTA  MTFCC FUNCSTAT    ALAND  AWATER     INTPTLAT  \
0      C5       N        N  G4110        A   777818       0  +39.7362744   
1      C5       N        N  G4110        A   731074       0  +40.9903610   
2      C5       N        N  G4110        A  1966413    8303  +39.9199958   
3      C5       N        N  G4110        A  7513072       0  +41.6272796   
4      C5       N        N  G4110        A  7996811       0  +41.1955437   

       INTPTLON                                         

/home/doctorgoose/miniconda3/envs/PAMJ/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


KeyError: 'zip_code'